# Evaluation

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, roc_curve, confusion_matrix
)

In [ ]:
# Example paths for predicted results and true labels
files = {
    "Logistic Regression": "logistic_regression_predictions.csv",
    "Random Forest": "random_forest_predictions.csv",
    "XGBoost": "xgboost_predictions.csv"
}

# Read predictions into a dictionary of DataFrames
results = {
    model: pd.read_csv(file)
    for model, file in files.items()
}

# Each file should have `y_true` (true labels) and `y_pred` (predicted labels or probabilities)